In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import the opencv library
import cv2
import torch # 토치 추가
import numpy as np
import time

# define a video capture object

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/lastbest.pt')  # local model
filepath = '/content/drive/MyDrive/파이썬 프로젝트/League of Legends (TM) Client 2023-05-02 20-50-28.mp4'
video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

if not video.isOpened():
    print("Could not Open :", filepath)
    exit(0)
    
#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

fourcc=cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer=cv2.VideoWriter('/content/resultf/08_result.mp4', fourcc, fps, (width, height))

blue_color=(255,0,0)
other_color=(0, 0, 255)
use=False # e 스킬 사용했는지 확인!
sucess=False # e 스킬 성공했는지 확인!
f=24 # 60 프레임 보겠다!
count_good=0 # 성공 횟수!
count_bad=0 # 실패 횟수!
count_frame=0 # 3번 동안 성공 유지했는지 보겠다!
frame_index=0

#def im_trim(img, x, y, w, h):
#    imgtrim = img[y: y + h, x: x + w]
#    return imgtrim

while(video.isOpened() and frame_index<length): # 
  frame_index+=1
  #print("start time : ", time.time())
    # Capture the video frame
   # by frame
  ret, frame = video.read()
  if(type(frame)==type(None)):
    continue
###   frame=frame[:240,0:320]

  # frame=im_trim(image, 680,326,1454,756)

  results=model(frame) # 만든 모델로 frame 분석!
  for i in range(len(results.pandas().xyxy[0].xmin)):
    if(results.pandas().xyxy[0].name[i]=="1" and results.pandas().xyxy[0].confidence[i]>=0.8): # 스킬 발동했다면
      frame=cv2.rectangle(frame, (int(results.pandas().xyxy[0].xmin[i]), int(results.pandas().xyxy[0].ymin[i])), (int(results.pandas().xyxy[0].xmax[i]), int(results.pandas().xyxy[0].ymax[i])), blue_color, 3)
      cv2.putText(frame, "use "+str(results.pandas().xyxy[0].confidence[i]), (int(results.pandas().xyxy[0].xmin[i]), int(results.pandas().xyxy[0].ymin[i])), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
      if(use==False):
          print("시비르 E 발동 시간 : ", frame_index/fps)
          use=True # use를 True로 변경!
      
    if(use==True): # 만약, use가 True라면
      if(f>0): # f(프레임수)가 0보다 크다면
        f-=1 # f를 1 감소시키기기
        if(results.pandas().xyxy[0].name[i]=="0" and results.pandas().xyxy[0].confidence[i]>=0.8): # 만약, 성공했다면
          count_frame+=1 # 성공 프레임 1 증가하기
          if(count_frame>=3): # 성공 프레임이 3개라면
            frame=cv2.rectangle(frame, (int(results.pandas().xyxy[0].xmin[i]), int(results.pandas().xyxy[0].ymin[i])), (int(results.pandas().xyxy[0].xmax[i]), int(results.pandas().xyxy[0].ymax[i])), other_color, 3)
            cv2.putText(frame, "success "+str(results.pandas().xyxy[0].confidence[i]), (int(results.pandas().xyxy[0].xmin[i]), int(results.pandas().xyxy[0].ymin[i])), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            count_good+=1 # 성공 횟수 1 증가시키기
            f=24 # 프레임 세는 것도 초기화하기
            use=False # use를 False로 바꾸기
            count_frame=0
            print("시비르 E 성공 시간 : ", frame_index/fps)

      else:
        print("시비르 E 실패 시간 : ", frame_index/fps)
        f=24
        count_bad+=1
        use=False
        count_frame=0
        
  writer.write(frame)

# After the loop release the cap object
video.release()
writer.release()

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)
print("성공 : ", count_good)
print("실패 : ", count_bad)
print("성공률 : ", count_good/(count_good+count_bad))
print("게임 시간 : ", length/fps)


# Destroy all the windows
cv2.destroyAllWindows()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7088338 parameters, 0 gradients
Adding AutoShape... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.
시비르 E 발동 시간 :  26.779169566375046
시비르 E 실패 시간 :  29.76581880372765
시비르 E 성공 시간 :  38.82644008558387
시비르 E 성공 시간 :  51.712657019779385
시비르 E 발동 시간 :  52.04823558577406
시비르 E 실패 시간 :  52.92073985736021
시비르 E 발동 시간 :  67.95465961392163
시비르 E 실패 시간 :  70.80707742487638
시비르 E 발동 시간 :  97.25066842525675
시비르 E 실패 시간 :  98.2238462666413
시비르 E 발동 시간 :  126.64735080639026
시비르 E 실패 시간 :  128.1574543533663
시비르 E 성공 시간 :  131.31189287371623
시비르 E 성공 시간 :  161.84954237923165
시비르 E 성공 시간 :  161.9837738056295
시비르 E 성공 시간 :  162.0844473754279
시비르 E 성공 시간 :  162.21867880182577
시비르 E 성공 시간 :  162.31935237162418
시비르 E 성공 시간 :  162.42002594142258
시비르 E 성공 시간 :  162.52069951122098
시비르 E 성공 시간 :  162.99050950361354
시비르 E 성공 시간 :  163.2254144998098
시비르 E 성공 시간 :  163.79589806200076
시비르 E 성공 시간 :  163.93012948839862
시비르 E 성공 시간 :  164.0643609147965
시비르 E 성공 시간 :  165.54090660517306
시비르 E 성공 시간 :  165.67513803157092
시비르 E 성공 시간 :  16